In [7]:
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re
import math
import time

In [9]:
# Instala o GeckoDriver
GeckoDriverManager().install()

# Inicie o navegador Firefox
navegador = webdriver.Firefox()
navegador.get('https://www.google.com')

In [10]:
info_resultados = []

# Salve os nomes das opções
option_names = ['RJ - Rio de Janeiro']

In [11]:
# Para cada opção, localize-a novamente e clique nela
for option_name in option_names:
    print(option_name)
    
    navegador.get("https://www.cnj.jus.br/e-natjus/pesquisaPublica.php")
    
    navegador.find_element('xpath', '//*[@id="frmPesquisa"]/div[5]/div[5]/div/span/span[1]/span/span[2]/b').click()
    options2 = WebDriverWait(navegador, 120).until(
        EC.visibility_of_all_elements_located((By.XPATH, '//ul[@id="select2-txtNatResponsavel-results"]/li'))
    )
    
    for option in options2:
        if option.text == option_name:
            option.click()
            break
    
    navegador.find_element('xpath', '/html/body/div[1]/div[1]/form/div[4]/div[2]/div/select').click()
    navegador.find_element('xpath', '//*[@id="selTipoTecnologia"]/option[2]').click()
    
    navegador.find_element('xpath', '//*[@id="btnPesquisar"]').click()
    
    try:
        element = WebDriverWait(navegador, 300).until(
            EC.visibility_of_element_located((By.XPATH, '//*[@id="resultados"]'))
        )
              
        html = navegador.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        texto = soup.find('div', id='resultados').find('p').text.strip()
        resultado = re.search(r'de (\d+) resultados', texto)
        numero_resultados = resultado.group(1)
        print(numero_resultados)
    except NoSuchElementException:
        pass  # Continuar para a próxima tentativa se element_xpath_2 não for encontrado
    
    numero_maximo_paginas = math.ceil(int(numero_resultados)/50)
    
    for pagina_numero in range(1, numero_maximo_paginas + 1):
        element = WebDriverWait(navegador, 120).until(
            EC.visibility_of_element_located((By.XPATH, '//*[@id="tbody"]/tr[1]'))
        )

        # Obter o código HTML da página
        html = navegador.page_source

        # Criar um objeto BeautifulSoup para analisar o código HTML
        soup = BeautifulSoup(html, 'html.parser')
        lista_de_tr = soup.find('div', attrs={'id': 'resultados'}).find('table').find('tbody').find_all('tr')

        for tr in lista_de_tr:
            dados = tr.find_all('td')

            identificacao = dados[0].text
            data = dados[1].text
            tecnologia = dados[2].text
            CID = dados[3].text
            status = dados[5].text
            links = dados[6].find_all('a')
            linkSite = links[0].get("href").strip()
            linkSite = "https://www.cnj.jus.br/e-natjus/" + linkSite
            linkPDF = links[1].get("href").strip()
            linkPDF = "https://www.cnj.jus.br/e-natjus/" + linkPDF 

            info_resultados.append([identificacao, data, tecnologia, CID, status, linkSite, linkPDF])
            
        # Verificar se a página atual é menor ou igual ao número máximo de páginas definido
        if pagina_numero < numero_maximo_paginas:
            # Clicar no número de página correspondente
            numero_pagina = navegador.find_element('xpath', f'//*[@id="pagination"]/nav/ul/li[{pagina_numero + 1}]/a')
            numero_pagina.click()
            time.sleep(2)  # Aguardar o carregamento da próxima página
        else:
            break  # Sai do loop se atingir o número máximo de páginas definido

RJ - Rio de Janeiro
1414


In [12]:
resultados_df = pd.DataFrame(info_resultados)
resultados_df.columns = ['Id do parecer', 'Data', 'Tecnologia', 'CID', 'Status', 'Link do Site', 'Link do PDF']

agora = datetime.now().strftime("%Y%m%d%H%M%S")
nome_arquivo = "enatjus-RaspagemLista-RioDeJaneiro-Medicamento-" + agora + ".xlsx"
resultados_df.to_excel(nome_arquivo, index=False)